In [ ]:
!git clone https://github.com/Seung-hwanSong/Anomaly.git #코랩 사용

# [시계열 이상치 탐지 Part 2]
## LOF / Isolation Forest

##### jupyter notebook 단축키

- ctrl+enter: 셀 실행   
- shift+enter: 셀 실행 및 다음 셀 이동   
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)
- 함수 ( ) 안에서 shift+tab: arguments description. shift+tab+tab은 길게 볼 수 있도록

## 1. 모듈 불러오기


In [ ]:
''' 데이터 전처리 패키지 '''
import numpy as np
import pandas as pd

''' 기계학습 모델 구축 및 평가 패키지 '''
from sklearn.model_selection import train_test_split

''' 데이터 시각화 패키지 '''
import seaborn as sns
import matplotlib.pyplot as plt

## 2. 데이터 불러오기: NASA Bearing Dataset

- 데이터 description <br>
    - NASA Bearing Dataset은 NSF I/UCR Center의 Intelligent Maintenance System의 4개의 bearing에서 고장이 발생할 때까지 10분 단위로 수집된 센서 데이터이다. 본 데이터셋은 특정 구간에서 기록된 1-second vibration signal snapshots을 나타내는 여러 개의 파일로 구성되어 있다. 각 파일은 20 kHz 단위로 샘플링 된 20,480개의 data point를 포함하고 있으며, 각 파일의 이름은 데이터가 수집된 시간을 의미한다. 해당 데이터셋은 크게 3개의 데이터를 포함하고 있으며, 본 실습에서 사용하는 데이터는 bearing 1에서 outer race failure가 발생할 때까지 수집된 센서 데이터이다. <br><br>
- 변수
    - 센서 데이터: Bearing1, Bearing2, Bearing3, Bearing4 <br><br>
- 출처: https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/

<img src="https://i.imgur.com/dfFzn3H.jpg" width="600">

### Step1. 데이터 불러오기

In [ ]:
# 데이터 불러오기
data = pd.read_csv('/content/Anomaly/data/nasa_bearing_dataset.csv', index_col=0)
# data = pd.read_csv('./data/nasa_bearing_dataset.csv', index_col=0)

data.index = pd.to_datetime(data.index)
data.head()

In [ ]:
# 데이터 크기
print("데이터 크기 : ", data.shape)

### Step2. 데이터 Split

In [ ]:
X_train = data[data['data_type'] == 'train'].iloc[:, :4]

X_test = data[data['data_type'] == 'test'].iloc[:, :4]
y_test = data[data['data_type'] == 'test'].iloc[:, -2].values

In [ ]:
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.6, random_state=2024)

print("Training data shape:", X_train.shape)
print("Validation data shape:", X_valid.shape)
print("Test data shape:", X_test.shape)

## 3. Local Oulier Factor

>이상치 스코어를 산출할 떄 주변부 데이터의 밀도를 고려하고자 함

<img src="https://i.imgur.com/flXG1jT.jpg" width="450">

In [ ]:
# LOF 모듈 불러오기

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, auc

### Step 0. 데이터 정규화

In [ ]:
# train 데이터를 기반으로 train/validation/test 데이터에 대하여 standard scaling 적용 (평균 0, 분산 1) 
scaler = StandardScaler()
scaler = scaler.fit(X_train)

In [ ]:
lof_X_train = pd.DataFrame(scaler.transform(X_train), 
                           columns=X_train.columns, index=X_train.index)
lof_X_valid = pd.DataFrame(scaler.transform(X_valid), 
                           columns=X_valid.columns, index=X_valid.index)
lof_X_test = pd.DataFrame(scaler.transform(X_test), 
                          columns=X_test.columns, index=X_test.index)

### Step 1. Train 데이터 기반 모델 적합

- LocalOutlierFactor 설명
    - n_neighbors: neighbors 개수 (default = 20)
    - algorithm: nearest neighbors 계산 방법 ('auto', 'ball_tree', 'kd_tree', 'brute'} (default = 'auto')
    - metric: distance 계산 방법 (default = 'minkowski')
    - novelty: True (for novelty detection - no oulier in train data), False (for outlier detection - ouliers in train data) (default = False)

In [ ]:
# LOF 모델 적합
lof_model = LocalOutlierFactor(n_neighbors=5, novelty=True)
lof_model.fit(lof_X_train)

### Step 2. 적합된 모델을 기반으로 train/test 데이터의 anomaly score 도출

In [ ]:
# train/validation/test 데이터의 LOF score 도출
lof_train = - 1.0 * lof_model.score_samples(lof_X_train)
lof_valid = - 1.0 * lof_model.score_samples(lof_X_valid)
lof_test = - 1.0 * lof_model.score_samples(lof_X_test)

In [ ]:
# train/validation/test 데이터의 anomaly score 분포 시각화
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize = (12, 8))

sns.distplot(lof_train, bins=100, kde=True, color='blue', ax=ax1)
sns.distplot(lof_valid, bins=100, kde=True, color='green', ax=ax2)
sns.distplot(lof_test, bins=100, kde=True, color='red', ax=ax3)
ax1.set_title("Train Data")
ax2.set_title("Validation Data")
ax3.set_title("Test Data")

### Step 3. Threshold 탐색

In [ ]:
# threshold 탐색
# score의 min ~ max 범위를 num_step개로 균등 분할한 threshold에 대하여 best threshold 탐색 
def search_best_threshold(score, y_true, num_step):
    best_f1 = 0
    best_threshold = None
    for threshold in np.linspace(min(score), max(score), num_step):
        y_pred = threshold < score

        f1 = f1_score(y_true, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    print('Best threshold: ', round(best_threshold, 4))
    print('Best F1 Score:', round(best_f1, 4))
    return best_threshold

In [ ]:
# best threshold 도출
lof_best_threshold = search_best_threshold(lof_valid, y_valid, num_step=1000)

### Step 4. Best threshold를 기반으로 이상치 탐지 모형 평가

In [ ]:
# 최종 결과 도출
lof_scores = pd.DataFrame(columns=['score', 'anomaly'])
for date, score in zip([X_train.index, X_valid.index, X_test.index], [lof_train, lof_valid, lof_test]):
    lof_score = pd.DataFrame(index=date)
    lof_score['score'] = score
    lof_score['anomaly'] = lof_best_threshold < score
    lof_scores = lof_scores.append(lof_score)

In [ ]:
lof_scores = lof_scores.sort_index()
lof_scores.head()

In [ ]:
# anomaly score plot 도출
def draw_plot(scores, threshold):
    normal_scores = scores[scores['anomaly'] == False]
    abnormal_scores = scores[scores['anomaly'] == True]

    plt.figure(figsize = (12,5))
    plt.scatter(normal_scores.index, normal_scores['score'], label='Normal', c='blue', s=3)
    plt.scatter(abnormal_scores.index, abnormal_scores['score'], label='Abnormal', c='red', s=3)
    
    plt.axhline(threshold, c='green', alpha=0.7)
    plt.axvline(data.index[int(len(data) * 0.5)], c='orange', ls='--')
    plt.axvline(data.index[int(len(data) * 0.7)], c='orange', ls='--')
    
    plt.xlabel('Date')
    plt.ylabel('Anomaly Score')
    plt.legend()
    
    plt.show()

In [ ]:
# 전체 데이터의 anomaly score 확인
draw_plot(lof_scores, lof_best_threshold)

In [ ]:
# FRR, FAR, F1 score, AUROC 도출
def calculate_metric(y_true, y_pred):
    # FRR, FAR
    cm = confusion_matrix(y_true, y_pred, labels=[True, False])
    tp, fn, fp, tn = cm.ravel()
    frr = fp / (fp + tn)
    far = fn / (fn + tp) 
    
    # F1 Score
    f1 = f1_score(y_true, y_pred)
    
    # AUROC, IE
    fpr, tpr, thresholds = roc_curve(y_true, y_pred, pos_label=1)
    auroc = auc(fpr, tpr)
        
    return frr, far, f1, auroc

In [ ]:
# 평가 지표 결과 확인
frr, far, f1, auroc = calculate_metric(y_test, lof_best_threshold < lof_test)

print("**  FRR: {}  |  FAR: {}  |  F1 Score: {}  |  AUROC: {}"
      .format(round(frr, 4), round(far, 4), round(f1, 4), round(auroc, 4)))

----

## 4. Isolation Forest

>하나의 객체를 고립시키는 의사결정나무를 생성하여 이상치를 탐지하고자 함 <br>
>정상 데이터는 고립시키는데 많은 분기가 필요하지만, 이상치 데이터라면 상대적으로 적은 분기만으로 고립이 가능함

<img src="https://i.imgur.com/VVbACBB.jpg" width="700">

In [ ]:
# Isolation Forest 모듈 불러오기

from sklearn.ensemble import IsolationForest

### Step 1. Train 데이터 기반 모델 적합

- IsolationForest 설명
    - n_estimators: 트리 개수 (default = 100)
    - bootstrap: 데이터의 중복 사용 여부 (default = False)
    - max_samples: 데이터 샘플 중 선택할 샘플의 수 혹은 비율 ('auto': max_samples=min(256, n_samples)) (default = 'auto')

In [ ]:
if_model = IsolationForest(random_state=2024)
if_model.fit(X_train)

### Step 2. 적합된 모델을 기반으로 train/test 데이터의 anomaly score 도출

In [ ]:
# train/validation/test 데이터의 IF score 도출
if_train = - 1.0 * if_model.score_samples(X_train)
if_valid = - 1.0 * if_model.score_samples(X_valid)
if_test = - 1.0 * if_model.score_samples(X_test)

In [ ]:
# train/validation/test 데이터의 anomaly score 분포 시각화
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize = (12, 8))

sns.distplot(if_train, bins=100, kde=True, color='blue', ax=ax1)
sns.distplot(if_valid, bins=100, kde=True, color='green', ax=ax2)
sns.distplot(if_test, bins=100, kde=True, color='red', ax=ax3)
ax1.set_title("Train Data")
ax2.set_title("Validation Data")
ax3.set_title("Test Data")

### Step 3. Threshold 탐색

In [ ]:
# best threshold 도출
if_best_threshold = search_best_threshold(if_valid, y_valid, num_step=1000)

### Step 4. Best threshold를 기반으로 이상치 탐지 모형 평가

In [ ]:
# 최종 결과 도출
if_scores = pd.DataFrame(columns=['score', 'anomaly'])
for date, score in zip([X_train.index, X_valid.index, X_test.index], [if_train, if_valid, if_test]):
    if_score = pd.DataFrame(index=date)
    if_score['score'] = score
    if_score['anomaly'] = if_best_threshold < score
    if_scores = if_scores.append(if_score)

In [ ]:
if_scores = if_scores.sort_index()
if_scores.head()

In [ ]:
# 전체 데이터의 anomaly score 확인
draw_plot(if_scores, if_best_threshold)

In [ ]:
# 평가 지표 결과 확인
frr, far, f1, auroc = calculate_metric(y_test, if_best_threshold < if_test)

print("**  FRR: {}  |  FAR: {}  |  F1 Score: {}  |  AUROC: {}"
      .format(round(frr, 4), round(far, 4), round(f1, 4), round(auroc, 4)))

----